In [32]:
import sys
import os
import re
#sys.path = [p for p in sys.path if p.find('/opt/apps/software/') == -1]
from glob import glob
from IPython.display import display, HTML
%matplotlib widget
from matplotlib import pyplot as plt

from utils.constants import UNIVARIATE_ARCHIVE_NAMES as ARCHIVE_NAMES
from utils.constants import UNIVARIATE_DATASET_NAMES as DATASET_NAMES
from utils.utils import read_all_datasets, transform_labels, create_directory, run_length_xps, generate_results_csv, plot_epochs_metric
from utils.data_loading import get_multiple_data_cf, predict, shifted_zscore_cf, zscore, print_metric, tf_rmse, tf_pmse, tf_pmse_cf, rmse
import utils
from classifiers import inception

import numpy as np
import pandas as pd
import sklearn

# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

def tf_pmse(y_true, y_pred):
    d = tf.reduce_mean((y_true - y_pred)**2, axis=0)
    if idx == -1:
        return tf.sqrt(tf.reduce_mean(dp))
    else:
        return tf.sqrt(dp[idx])

def tf_pmse_DA(y_true, y_pred):
    return tf_pmse_cf(y_true, y_pred, idx=0)

def tf_pmse_5HT(y_true, y_pred):
    return tf_pmse_cf(y_true, y_pred, idx=1)

def tf_pmse_pH(y_true, y_pred):
    return tf_pmse_cf(y_true, y_pred, idx=2)

def tf_pmse_NE(y_true, y_pred):
    return tf_pmse_cf(y_true, y_pred, idx=3)

# import keras
# keras.backend.tensorflow_backend._get_available_gpus()

In [2]:
names = ['DA', '5HT', 'pH', 'NE']
speed = 'slow'
data_prefix = '/mnt/nfs/proj/in-vitro/Leonardo/cf_data'

probes = [
    'CF025', 'CF027', 'CF057', 'CF064', 'CF066', 'CF078', 'CF081', 'CF082'
]

hold_probe = probes[7]

# val_probe=None
val_probe=probes[0]

print(f'Leaving out probe {hold_probe}', flush=True)
print(f'Validation probe {val_probe}', flush=True)
print(f'Loading data', flush=True)

# normalize_data = minmax
# revert_data = lambda x: minmax(x, inverse=True)

normalize_data = shifted_zscore_cf
revert_data = lambda x: shifted_zscore_cf(x, inverse=True)

# normalize_data = lambda x: x
# revert_data = lambda x: x

# this is actually the number of records per UNIQUE CONCENTRATIONS per probe
n_records_per_probe = -1 # all
# n_records_per_probe = 1

x_train, y_train, x_val, y_val, x_test, y_test = get_multiple_data_cf(data_prefix,
                                                                      probes=probes,
                                                                      hold_probe=hold_probe,
                                                                      val_probe=val_probe,
                                                                      normalize_data=normalize_data,
                                                                      n_records_per_probe=n_records_per_probe)

print('Data loaded')

# output_shape = y_train.shape[1]

# if len(x_train.shape) == 2:  # if univariate
#     print('adding singleton')
#     # add a dimension to make it multivariate with one dimension
#     x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
#     x_val = x_val.reshape((x_test.shape[0], x_test.shape[1], 1))
#     x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))


Leaving out probe CF082
Validation probe CF025
Loading data
loading probe CF025
(61650, 999) (61650, 4)
loading probe CF027
(61650, 999) (61650, 4)
loading probe CF057
(61650, 999) (61650, 4)
loading probe CF064
(61650, 999) (61650, 4)
loading probe CF066
(61650, 999) (61650, 4)
loading probe CF078
(61650, 999) (61650, 4)
loading probe CF081
(61650, 999) (61650, 4)
loading probe CF082
(61650, 999) (61650, 4)
Shuffling training dataset
Data loaded


In [3]:

if len(x_train.shape) == 2:  # if univariate
    print('adding singleton')
    # add a dimension to make it multivariate with one dimension
    x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
    x_val = x_val.reshape((x_val.shape[0], x_val.shape[1], 1))
    x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))

output_shape = y_train.shape[1]
input_shape = x_train.shape[1:]

output_directory = f'/mnt/nfs/proj/in-vitro/Leonardo/inception/results/cf/{hold_probe}/'

if not (os.path.exists(output_directory)):
    os.makedirs(output_directory, exist_ok=True)


adding singleton


In [4]:
for x in [x_train, y_train, x_val, y_val, x_test, y_test]:
    print(x.shape)
print(output_shape)
print(input_shape)

(369900, 999, 1)
(369900, 4)
(61650, 999, 1)
(61650, 4)
(61650, 999, 1)
(61650, 4)
4
(999, 1)


In [5]:
# resnet model
import keras
import numpy as np
import time

from utils.utils import save_logs
from utils.utils import calculate_metrics
from utils.utils import save_test_duration

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

def tf_rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean((y_true - y_pred)**2))


In [6]:
# Fit the model

classifier_name = 'inception'

classifier = inception.Regression_INCEPTION(output_directory, input_shape, output_shape, verbose=1, build=True, nb_epochs=500)
# classifier = Regression_INCEPTION(output_directory, input_shape, output_shape, verbose=1, build=True, nb_epochs=100)

metrics = classifier.fit(x_train, y_train, x_val, y_val, plot_test_acc=True)

display(HTML(metrics.to_html()))


mini batch size: 64
Train on 369900 samples, validate on 61650 samples
Epoch 1/100
369900/369900 [==============================] - 332s 897us/step - loss: 0.3839 - tf_pmse_DA: 488.6883 - tf_pmse_5HT: 418.8771 - tf_pmse_pH: 0.0728 - tf_pmse_NE: 545.8939 - val_loss: 0.9853 - val_tf_pmse_DA: 879.1813 - val_tf_pmse_5HT: 389.8340 - val_tf_pmse_pH: 0.1891 - val_tf_pmse_NE: 1166.5785
Epoch 2/100
369900/369900 [==============================] - 322s 869us/step - loss: 0.0586 - tf_pmse_DA: 280.8677 - tf_pmse_5HT: 240.6985 - tf_pmse_pH: 0.0390 - tf_pmse_NE: 297.6361 - val_loss: 0.2302 - val_tf_pmse_DA: 334.7719 - val_tf_pmse_5HT: 479.0947 - val_tf_pmse_pH: 0.0672 - val_tf_pmse_NE: 466.7991
Epoch 3/100
369900/369900 [==============================] - 321s 869us/step - loss: 0.0402 - tf_pmse_DA: 233.9248 - tf_pmse_5HT: 194.7941 - tf_pmse_pH: 0.0323 - tf_pmse_NE: 245.9588 - val_loss: 0.3612 - val_tf_pmse_DA: 577.6031 - val_tf_pmse_5HT: 534.7583 - val_tf_pmse_pH: 0.0916 - val_tf_pmse_NE: 547.2230
E

,rmse_DA,rmse_5HT,rmse_pH,rmse_NE,duration
0,483.575696,504.289784,0.072682,604.31947,32320.917521


In [7]:
# Hold CF082, validation CF025, all data, 100 epochs
# Epoch 97/100
# 369900/369900 [==============================] - 323s 872us/step - loss: 0.0021 - tf_pmse_DA: 50.4458 - tf_pmse_5HT: 46.5694 - tf_pmse_pH: 0.0077 - tf_pmse_NE: 54.5813 - 
# val_loss: 0.1685 - val_tf_pmse_DA: 356.7854 - val_tf_pmse_5HT: 305.9508 - val_tf_pmse_pH: 0.0534 - val_tf_pmse_NE: 400.9060
# predicting validation set... 
# 	rmse_DA 	rmse_5HT 	rmse_pH 	rmse_NE 	duration
# 0 	483.575696 	504.289784 	0.072682 	604.31947 	32320.917521

In [15]:
metrics = classifier.predict(x_test, y_test, x_train, y_train, return_df_metrics=True)
display(HTML(metrics.to_html()))

# Hold CF082, validation CF025, all data, 100 epochs
# rmse_DA 	rmse_5HT 	rmse_pH 	rmse_NE 	duration
# 289.592247 	148.701677 	0.082032 	416.473678 	0.0

,rmse_DA,rmse_5HT,rmse_pH,rmse_NE,duration
0,289.592247,148.701677,0.082032,416.473678,0.0


In [51]:
print(classifier.output_directory)

# model_path = os.path.join(classifier.output_directory, 'best_model.hdf5')
model_path = os.path.join(classifier.output_directory, 'last_model.hdf5')
# model_path = os.path.join(classifier.output_directory, 'model_init.hdf5')

model = keras.models.load_model(model_path, custom_objects={"tf_pmse_DA": tf_pmse_DA, "tf_pmse_5HT": tf_pmse_5HT, "tf_pmse_pH": tf_pmse_pH, "tf_pmse_NE": tf_pmse_NE})

xt, yt = x_test, y_test
# xt, yt = x_val, y_val

yp = model.predict(xt, batch_size=64)

yp = np.apply_along_axis(revert_data, axis=1, arr=yp) 
yt = np.apply_along_axis(revert_data, axis=1, arr=yt) 

rmse4 = rmse(yt, yp)

metrics2 = pd.DataFrame(data=np.zeros((1, 5), dtype=np.float), index=[0], columns=['rmse_DA', 'rmse_5HT', 'rmse_pH', 'rmse_NE', 'duration'])
metrics2['rmse_DA'] = rmse4[0]
metrics2['rmse_5HT'] = rmse4[1]
metrics2['rmse_pH'] = rmse4[2]
metrics2['rmse_NE'] = rmse4[3]
metrics2['duration'] = 0.0
        
display(HTML(metrics2.to_html()))


/mnt/nfs/proj/in-vitro/Leonardo/inception/results/cf/CF082/


,rmse_DA,rmse_5HT,rmse_pH,rmse_NE,duration
0,289.592247,148.701677,0.082032,416.473678,0.0


In [52]:
# def plot_concentrations(y):
#     fig, axs = plt.subplots(2, 2, sharey=False, tight_layout=True)
#     for ip in range(0,4):
#         axs[np.unravel_index(ip, axs.shape)].hist(y[:,ip])
# plot_concentrations(y_test)
# plot_concentrations(y_pred)

def plot_compare_test_pred(yt, yp):
    fig, axs = plt.subplots(2, 4, sharey=False, tight_layout=True, figsize=(12, 5))
    for ip in range(0,8):
        axsidx = np.unravel_index(ip, axs.shape)
        if ip < 4:
            axs[axsidx].hist(yt[:,ip])
        else:
            axs[axsidx].hist(yp[:,ip-4])

plot_compare_test_pred(yt, yp)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …